## Data Prep 

In [32]:
import pandas as pd

### Calling our Helper Functions 

In [33]:
# add the scripts directory to the python path, or the set of places the python interpreter looks for code files
import sys
sys.path.insert(0, './scripts')

from helpers import read_in_dataset

In [34]:
import inspect
print(inspect.getsource(read_in_dataset))

def read_in_dataset(dset, verbose=False):
    
    """Read in one of the Zillow datasets (train or properties)
    
    Keyword arguments:
    dset -- a string in {properties_2016, properties_2017, train_2016, train_2017}
    verbose -- whether or not to print info about the dataset
    
    Returns:
    a pandas dataframe
    """
    from pandas import read_csv
    
    df = read_csv('raw_data/{0}.csv'.format(dset))
    
    if verbose:
        print("\nReading in the {0} dataset ".format(dset))
        print('\nit has {0} rows and {1} columns'.format(df.shape[0], df.shape[1]))
        print("\nIt has the folowing columns ")
        print(df.columns)
        print("\nThe first 5 rows look like this ")
        print(df.head(5))
        
    return df



In [35]:
## using ipython
read_in_dataset??

## Read in the Data 

In [36]:
# Loading DataGuide for reference
dataguide = pd.read_excel("raw_data/zillow_data_dictionary.xlsx")
dataguide.head()

,Feature,Description
0,'airconditioningtypeid',Type of cooling system present in the home (i...
1,'architecturalstyletypeid',"Architectural style of the home (i.e. ranch, ..."
2,'basementsqft',Finished living area below or partially below...
3,'bathroomcnt',Number of bathrooms in home including fractio...
4,'bedroomcnt',Number of bedrooms in home


#### Properties

This dataset has information about the properties we'll be trying to make predictions on.

In [37]:
properties = read_in_dataset('properties_2016', verbose=True)

C:\Users\sac\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):



Reading in the properties_2016 dataset 

it has 2985217 rows and 58 columns

It has the folowing columns 
Index(['parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytyp

#### Target Variable and Transaction Date

This dataset includes our Target (or, response, or dependent) variable as well as a key predictor (or feature), transactiondate.

In [38]:
train_raw = read_in_dataset("train_2016", verbose=True)


Reading in the train_2016 dataset 

it has 90275 rows and 3 columns

It has the folowing columns 
Index(['parcelid', 'logerror', 'transactiondate'], dtype='object')

The first 5 rows look like this 
   parcelid  logerror transactiondate
0  11016594    0.0276      2016-01-01
1  14366692   -0.1684      2016-01-01
2  12098116   -0.0040      2016-01-01
3  12643413    0.0218      2016-01-02
4  14432541   -0.0050      2016-01-02


## Explore the Raw Datasets 

In [39]:
# how many train records don't have a matching property
len(set(train_raw.parcelid) - set(properties.parcelid))

0

In [40]:
# how many properties don't have a matching train record
len(set(properties.parcelid) - set(train_raw.parcelid))

2895067

So there are a lot of properties that don't have a corresponding target. What could we use these records for? For now, lets separate them out so we don't have to carry them around in memory

## Make Dataset for Analysis

There's a lot of data in the __properties__ dataset that we won't be using (because those properties didn't sell and therefore we don't know what the target values are).To improve the speed of loading in the data and reduce the memory strain, let's create a file with only the properties and sales records we'll need for training.

In [41]:
from helpers import merge_dataset

In [42]:
print(inspect.getsource(merge_dataset))

def merge_dataset(train, properties):
    
    """Merge the train and properties datasets. Both have a common key 'parcelid'.
    
    Keyword arguments:
    train -- the dataframe of transactions
    properties -- the dataframe of properties
    
    Returns:
    a pandas dataframe
    """
    
    
    train_data_merged = train.merge(properties, how="left", on="parcelid")
    
    return train_data_merged 



In [52]:
# merge the data and remove the raw datasets

train_data_merged = merge_dataset(train_raw, properties)
del train_raw
del properties

In [53]:
train_data_merged.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13


Now this is what we'll call our modeling file because it has all the data needed to build the model.Let's put it in a directory called derived_data so we can read it in in the next notebook. 

In [54]:
train_data_merged.to_csv("derived_data/train_data_merged_2016.csv", index=False)